## 準備

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.optimize import curve_fit

In [ ]:
def fit_func(x, count, mu, sigma, a, b):
    return count * np.exp(- (x - mu)**2 / (2 * sigma**2)) / (np.sqrt(2. * np.pi) * sigma) + a + b * x

## データの読み込み

In [ ]:
# input the text file name
file_name = "Ba.txt"

raw_data = np.loadtxt(file_name)
ch = np.arange(4096)

plt.plot(ch, raw_data)
plt.savefig("py_raw_data.png") # save
plt.show()

## ピークサーチ

In [ ]:
# 範囲の指定
data_range = (80, 1100)

data = raw_data[data_range[0]:data_range[1]]
ch = np.arange(data_range[0], data_range[1])

# parameters: height, threshold, distance, prominence
peaks, results = find_peaks(data, height=10, prominence=50)
pos_peaks = peaks + data_range[0]
height_peaks = results["peak_heights"]

plt.plot(ch, data, label="data")
plt.plot(ch[peaks], data[peaks], "x", label="find peaks")
plt.yscale("log")
plt.legend()
plt.savefig("py_find_peaks.png") # save
plt.show()

## フィッティング

In [ ]:
x_list = []
y_list = []

for i in range(len(peaks)):
    fine_range = (pos_peaks[i] - 15, pos_peaks[i] + 15)
    fine_data = raw_data[fine_range[0]:fine_range[1]]
    fine_ch = np.arange(fine_range[0], fine_range[1])

    # initial parameters
    par, cov = curve_fit(fit_func, fine_ch, fine_data, sigma=np.sqrt(fine_data), p0=[height_peaks[i]*2.0, pos_peaks[i], 1.0, 10.0, -0.001], absolute_sigma=True)
    chi2 = np.sum(((fit_func(fine_ch, par[0], par[1], par[2], par[3], par[4]) - fine_data)/np.sqrt(fine_data))**2)
    ndf = len(fine_ch) - 5

    plt.plot(fine_ch, fine_data, 'o', label="data")
    x = np.arange(fine_range[0], fine_range[1], 0.01)
    x_list.append(x)
    y_list.append(fit_func(x, par[0], par[1], par[2], par[3], par[4]))
    plt.plot(x, fit_func(x, par[0], par[1], par[2], par[3], par[4]), label="fitting")
    plt.yscale("log")
    plt.legend()
    plt.grid()
    plt.savefig("py_fit" + str(i) + "_result.png") # save
    plt.show()

    print("chi2/ndf = {:7.3f}/{}".format(chi2, ndf))
    print("p0 : {:10.5f} +- {:10.5f}".format(par[0], np.sqrt(cov[0,0])))
    print("p1 : {:10.5f} +- {:10.5f}".format(par[1], np.sqrt(cov[1,1])))
    print("p2 : {:10.5f} +- {:10.5f}".format(par[2], np.sqrt(cov[2,2])))
    print("p3 : {:10.5f} +- {:10.5f}".format(par[3], np.sqrt(cov[3,3])))
    print("p4 : {:10.5f} +- {:10.5f}".format(par[4], np.sqrt(cov[4,4])))

In [ ]:
plt.plot(ch, data)
for i in range(len(x_list)):
    plt.plot(x_list[i], y_list[i])
plt.yscale("log")
plt.savefig("py_fit_all.png") # save
plt.show()